In [1]:
import pandas as pd
import glob
import os
import xlwings as xw
from xlwings.constants import LineStyle
from xlwings import utils
from xlwings import Range

from tkinter import *
from tkinter import ttk
from tkinter import filedialog
import datetime

import shutil
import tempfile
import pyperclip
import sys

C:\Users\l.nguyen quoc\Anaconda3\lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
# Initialisierung Datum
heute = datetime.date.today()
heute_datum = heute.strftime("%d.%m.%Y")

# Funktion, um Zwischenablage zu löschen
def clear_windows_clipboard():
    #os.system("echo off | clip")
    pyperclip.copy('')
    
    
# Funktion, welches Makro ausführt, um Dropdown zu erstellen
def create_dropdown(wb, dropdown_liste, zelle):
    wb.macro("CreateDropdown")(dropdown_liste, zelle)


In [3]:
# Einlesen Datenbank-Tabelle
try:
    data_path = r'O:\Serumbank\Wareneingang\Vorlagen'
    files = glob.glob(os.path.join(data_path, '*.xlsx*'))
    for file in files:
        if ('Datenbanktool' in file) & ('Kopie' not in file):
            data_path = file
            break
            
    # Kopie der Datenbank erstellen
    datenbank_copy = "Datenbank_Copy.xlsx"
    shutil.copyfile(data_path, datenbank_copy)
            
    # Datenbank mit pandas einlesen            
    datenbank_pd = pd.read_excel("Datenbank_Copy.xlsx", sheet_name = 'Datenbank-Spendeneingang', header = 1)
    
    # gleichzeitig Produktnummern ablesen
    produkt_nr = pd.read_excel("Datenbank_Copy.xlsx", sheet_name = 'Produktnummern')
    
    data_prim_id = []
    for i in range(len(datenbank_pd)):
        data_prim_id.append(datenbank_pd.iloc[i,0])
    # Datenbank nach Primär ID splitten und Tabellen in dictionary speichern
    # Liste mit Start-Index für jede Tabelle
    start_indices = []
    for i in range(len(data_prim_id)):
        if (str(data_prim_id[i]).lower() != 'x') & (data_prim_id[i] != 'None'):
            start_indices.append(i+3)
    table_ranges = []
    for i in range(len(start_indices)-1):
        table_ranges.append('C{}:AE{}'.format(start_indices[i], start_indices[i+1]-1))

except:
    root = Tk()
    root.title('AutoSpendenankunft')
    root.geometry('400x200')
    root.iconbitmap('O:\Forschung & Entwicklung\Allgemein\Vorlagen\Abbildungen\Dodo\dodo_icon.ico')
    root.config(bg='#eeeee4')
    
    text_datenbank = Label(root, text = "\n  'Datenbank' ist von jemandem geöffnet.\n  Bitte schließen und Programm erneut starten.\n",
                     bg = '#eeeee4', font=('Ink free',12, 'bold'))
    text_datenbank.grid(row=1, column=0)  
        
    # Logo
    from PIL import ImageTk, Image
    frame = Frame(root, width=1, height=1)
    frame.grid(row=2, column=0)

    img = ImageTk.PhotoImage(Image.open("O:\Forschung & Entwicklung\Allgemein\Vorlagen\Abbildungen\Dodo\dodo-dancing_ohne Hintergrund_ohne Schatten.png").resize((70,70)), master = root)
    label = Label(frame, image = img, bg = '#eeeee4')
    label.pack()

    root.mainloop()

In [4]:
# Funktion zum Initialisieren
def init_id_aliquot():
    global aliquot_column, id_vergabe
    column = 'B'
    last_row = sht1.range(column + str(sht1.cells.last_cell.row)).end('up').row
    id_vergabe_range = sht1.range(column + '1:' + column + str(last_row))
    id_vergabe = id_vergabe_range.value
    id_vergabe = [entry for entry in id_vergabe if entry is not None]
    column_range = sht1.range('X{}:X{}'.format(3, len(id_vergabe)+1))
    aliquot_column = column_range.value


# Öffnen der Spendenankunft-Tabelle zum Bearbeiten
def browse():
    global text_browse
    try:
        text_browse.grid_forget()
    except:
        pass
        
    try:
        global path, spendenankunft, sht1, sht_drop, start_id, end_id
        path = filedialog.askopenfilename(title = "Datei auswählen")
        # Erstellen der txt-Datei für fill_excel.py
        file_path = os.getcwd()
        with open(file_path + '\\' + "path.txt", "w") as f:
            f.write(path)             
        
        spendenankunft = xw.Book(path)        
        sht1 = spendenankunft.sheets['Spendenankunft']
        sht_drop = spendenankunft.sheets['Dropdown']

        # Initialisierung von später benötigten Variablen
        # Aliquot-Spalte ablesen und Liste mit allen vergebenen IDs erstellen
        init_id_aliquot()
        
        # Ablesen der Start und End Master ID zum späteren Check, ob diese manuell geändert wurden
        # Erstes Ablesen dient der Initialisierung
        start_id = sht1.range('C1').value
        end_id = sht1.range('E1').value
    # Falls nichts ausgewählt wurde
    except:
        text_browse = Label(root, text = "\nBitte wähle eine Spendenankunft-Tabelle aus.", fg='#CD5C5C',
                     bg = '#eeeee4', font=('Ink free',12, 'bold'))
        text_browse.grid(row=9, column=1, columnspan=10)
        spendenankunft.close()

        
def go_dodo():
    """ Funktion für User Interface, um Excel auszufüllen
    """
    
    global spendenankunft, sht1, red_flag_middle, red_flag, scan_text, master_id_error, type_error_comp, aliquot_error, barcode_error, data_error, datenbank_error, id_error, id_check_error, end_id_error

    # red_flag initialisieren (gibt an, wenn Master ID überschritten wurde)
    red_flag = False
    red_flag_middle = False
    
    # Schritt 1: Über Scannen des Barcodes Spendenankunft erstellen
        # Ablesen der generierten Primär ID aus Spendenankunft-Tabelle
        # Suchfunktion basierend auf der Projektnummer in Datenbank-Tabelle
        # Ausfüllen der Spendenankunft-Tabelle
    try:
        aliquot_error.grid_forget()
    except:
        pass
        
    try:
        opened_file.grid_forget()
    except:
        pass
    
    try:
        data_error.grid_forget()
    except:
        pass
    
    try:
        datenbank_error.grid_forget()
    except:
        pass
    
    try:
        id_error.grid_forget()
    except:
        pass
    
    try:
        id_check_error.grid_forget()
    except:
        pass
    
    try:
        end_id_error.grid_forget()
    except:
        pass
    
    try:
        text_error.grid_forget()
    except:
        pass
    
    try:
        type_error_comp.grid_forget()
    except:
        pass
    
    try:
        barcode_error.grid_forget()
    except:
        pass
    
    try:
        scan_text.grid_forget()
    except:
        pass
    
    try:
        master_id_error.grid_forget()
    except:
        pass
    
    # nach Zeile mit roter Primär ID suchen
    # den Schritt skippen, wenn keine Einträge vorhanden sind
    if sht1.range('B3').value is not None:
        last_row_excel = sht1.range('B3').end('down').row
        for i in range(3, last_row_excel):
            cell = sht1.range('B{}'.format(i))
            if (cell.api.Font.Color == xw.utils.rgb_to_int((255, 0, 0))) | (cell.api.Font.Color == xw.utils.rgb_to_int((255, 165, 0))):
                start_row_flag = i
                red_flag_middle = True
                break
    
    def fill_excel():
        """ Füllt Excel-Tabelle aus
        """           
        # aktuelle Start-Zeile identifizieren
        # (wird daran erkannt, dass Zelle in Spalte 'Primär ID' leer ist)
        global red_flag, last_row, start_row, data_index, data_index_end, data_error, datenbank_error, id_error, id_check_error, end_id_error, type_error_comp, master_id_error  
        
        if red_flag:
            # Wenn im untersuchten Block alle IDs rot sind, wird der gesamte Block ersetzt
            if start_row == data_index:
                start_row = last_row
        elif red_flag_middle:
            start_row = start_row_flag        
        else:
            start_row = 3
            while True:        
                if sht1.range('B{}'.format(start_row)).value is None:
                    break
                else:
                    start_row +=1
                    
        prim_id_total = sht1.range('B1').value
        
        # für den Check, ob End-Master ID bereits vergeben wurde
        master_id_given_col = sht1.range('B3:B{}'.format(last_row_excel)).value
        master_id_given = [master for master in master_id_given_col if master is not None]
        # Liste mit vergebenen Master IDs, die nicht rot sind
        column_range = sht1.range('B3:B{}'.format(last_row_excel))
        entries_in_black = []
        for cell in column_range:
            # falls es sich um den ersten Eintrag handelt:
            if str(cell.value) == 'None':
                break
            # Check if the font color is black (RGB value for black: 0)
            font_color = cell.api.Font.Color
            if font_color == 0:
                entries_in_black.append(cell.value)
        master_id_given = entries_in_black
            
        
        if red_flag_middle:
            new_end_id = sht1.range('E1').value
            
            # Check, ob Barcode-Nr. bereits gescannt wurde            
            scanned_barcodes = sht1.range('C3:C{}'.format(last_row_excel)).value    
            scanned_barcodes = list(set([barcode for barcode in scanned_barcodes if barcode is not None]))
            
             # falls Master ID überschritten und nicht verändert wurde, poppt User Warnung auf
            if new_end_id in master_id_given:
                # falls Master ID überschritten und neue Probe eingescannt wurde
                if prim_id_total not in scanned_barcodes:
                    root_master = Frame(root, width=50, height=50, bg = '#eeeee4')
                    root_master.grid(row=2, column=1, columnspan=50, rowspan=50)
                    warning_text = Label(root_master, text = '       Master ID überschritten und neue Probe eingescannt        \n',
                                     bg = '#eeeee4', font=('Ink free',12))
                    warning_text.grid(row=0, column=0, columnspan=15)
                    def warning_back():
                        root_master.grid_forget()
                    nein = Button(root_master, text='Zurück', command=warning_back, font=('Ink free',10,'bold'), bg='#869287')
                    nein.grid(row=1, column=5)
                    space = Label(root_master, text = '\n\n\n\n\n\n\n\n\n\n', bg = '#eeeee4')
                    space.grid(row=2, column=1)
                else:
                    root_master = Frame(root, width=50, height=50, bg = '#eeeee4')
                    root_master.grid(row=2, column=1, columnspan=50, rowspan=50)
                    warning_text = Label(root_master, text = '       Master ID überschritten, bitte neue vergeben        \n',
                                     bg = '#eeeee4', font=('Ink free',12))
                    warning_text.grid(row=0, column=0, columnspan=15)
                    def warning_back():
                        root_master.grid_forget()
                    nein = Button(root_master, text='Zurück', command=warning_back, font=('Ink free',10,'bold'), bg='#869287')
                    nein.grid(row=1, column=5)
                    space = Label(root_master, text = '\n\n\n\n\n\n\n\n\n\n', bg = '#eeeee4')
                    space.grid(row=2, column=1)
                
            else:
                if prim_id_total in scanned_barcodes:
                    # es werden dann nur die roten Master IDs mit den neuen ersetzt
                    # neue end ID ablesen
                    old_id_list = sht1.range('B{}:B{}'.format(start_row, last_row_excel)).value
                    # Finden der neuen start id
                    new_start_id = sht1.range('C1').value
                    while True:
                        if new_start_id not in master_id_given:
                            break
                        new_start_id += 1

                    for i in range(len(old_id_list)):
                        cell_update = sht1.range('B{}'.format(start_row+i))
                        if i == 0:
                            old_id = sht1.range('B{}'.format(start_row+i)).value                       
                            if cell_update.value == old_id:
                                if cell_update.api.Font.Color == xw.utils.rgb_to_int((255, 0, 0)):
                                    cell_update.api.Font.Color = xw.utils.rgb_to_int((0, 0, 0))

                                if new_start_id > sht1.range('C1').value:
                                    cell_update.value = new_start_id
                                else:
                                    cell_update.value = sht1.range('C1').value
                                    
                        else:
                            if sht1.range('C{}'.format(start_row+i)).value is not None:
                                new_start_id += 1
                            if cell_update.api.Font.Color == xw.utils.rgb_to_int((255, 0, 0)):
                                
                                if new_start_id > sht1.range('C1').value:
                                    cell_update.value = new_start_id
                                else:
                                    cell_update.value = sht1.range('C1').value

                                old_id = sht1.range('B{}'.format(start_row+i)).value
                                cell_update.api.Font.Color = xw.utils.rgb_to_int((0, 0, 0))
                        # falls ID wieder überschritten wird:
                        if new_start_id > new_end_id:
                            cell_update.api.Font.Color = xw.utils.rgb_to_int((255, 0, 0))
                            

        else:            
            # Suchfunktion
            # Projektnr. ablesen
            flag_yellow = False
            if len(prim_id_total.split('-')) == 3:
                projektnr = prim_id_total.split('-')[1]
            else:
                projektnr = prim_id_total.split('-')[-1]
                flag_yellow = True
            # Suche in Datenbank und Index des entsprechenden Eintrags ablesen
            data_index = None
            for i in range(len(data_prim_id)):
                if projektnr in data_prim_id[i]:
                    data_index = i
                    break
            for i in range(data_index+1,len(data_prim_id)):
                if data_prim_id[i].upper() == 'X':
                    data_index_end = i
                else:
                    break

            # Check, ob Primär ID-Feld orange ist
            # Datenbank mit xlwings einlesen
            app = xw.App(visible=False)    
            datenbank = xw.Book("Datenbank_Copy.xlsx")
            data_sheet = datenbank.sheets['Datenbank-Spendeneingang']
            
            if (data_sheet.range('A{}'.format(3+data_index)).color == (255, 153, 0)) | (data_sheet.range('A{}'.format(3+data_index)).color == (255, 192, 0)):
                data_error = Label(root, text = "\nSpendendatensatz noch nicht freigegeben", fg='#CD5C5C',
                                    bg = '#eeeee4', font=('Ink free',12, 'bold'))
                data_error.grid(row=9, column=1, columnspan=10)
            else:
                data_table = data_sheet.range('C{}:AE{}'.format(data_index+3, data_index_end+3))
                data_table.api.Copy()
                start_cell = sht1.range('D{}'.format(start_row))
                # Copy and paste the table data and formatting
                data_table.api.Copy()
                start_cell.api.PasteSpecial(Paste=-4163)  # -4163 corresponds to Paste All             
                clear_windows_clipboard()

                # Spalte Primär ID leer und grau
                empty_id_cells = sht1.range('C{}:C{}'.format(start_row+1, data_index_end-data_index+start_row))
                empty_id_cells.value = ''
                empty_id_cells.color = (174, 170, 170)       

                # Eintragen von Datum    
                datum_index = []
                for j in range(data_index_end-data_index+1):
                    cell_date = sht1.range('F' + str(start_row+j))
                    if (cell_date.color == (255, 255, 0)) | (cell_date.color != (174, 170, 170)):
                        cell_date.value = heute_datum
                        datum_index.append(j)
                einlagerung_cols = ['L', 'O', 'Q', 'S']
                update_ranges = []

                for col in einlagerung_cols:
                    for j in datum_index:
                        cell_date = sht1.range(col + str(start_row + j))
                        if cell_date.value != '/':
                            update_ranges.append(cell_date)

                # Set the values and colors in bulk
                for cell_date in update_ranges:
                    cell_date.value = heute_datum

                # Spalte mit Projektnummer ausfüllen
                # beim Start einer neuen Master ID, trägt die Zeile eine Projektnr.
                # restl. Zellen darunter werden mit '/' gefüllt

                # Überprüfen, ob Produktnr vorhanden ist
                list_projektnr = list(produkt_nr['Projektnummer'])
                list_produktnr = list(produkt_nr['Produktnummer'])

                for i in range(len(produkt_nr)):
                    if projektnr in list_projektnr[i]:
                        projektnr = projektnr + '\n' + list_produktnr[i]
                        break


                ziffer_flag = False
                for i_nr in range(len(projektnr)):
                    if projektnr[i_nr] in ['1', '2', '3', '4', '5', '6', '7', '8', '9']:
                        ziffer_flag = True
                        break
                if ziffer_flag:
                    # Projektnr. wird in der gesamten Spalte eingetragen
                    cell_nr = sht1.range('Z{}:Z{}'.format(start_row, start_row+data_index_end-data_index))
                    cell_nr.number_format = 'Standard'
                    cell_nr.value = str(projektnr)
                    cell_nr.color = (255, 255, 255)

                # Operator-Spalte ausfüllen
                operator = sht1.range('F1').value
                cell_ops = [sht1.range('W{}'.format(i + start_row)) for i in datum_index]
                cell_prims = [sht1.range('C{}'.format(i + start_row)) for i in datum_index]

                for cell_op in cell_ops:
                    cell_op.value = operator
                    cell_op.color = (255, 255, 255)
                    cell_op.api.Font.ColorIndex = 1

                for cell_prim in cell_prims:
                    cell_prim.value = prim_id_total
                    if flag_yellow:
                        cell_prim.color = (255, 255, 0)
                    else:
                        cell_prim.color = (255, 255, 255)
                    cell_prim.api.Font.Bold = True

                # LID-Spalte ergänzen, falls Zelle nicht grau ist
                for i in range(data_index_end-data_index+1):
                    lid_cell = sht1.cells(start_row+i, 4)
                    lid = prim_id_total.split('-')[0]    
                    try:
                        lid = int(lid)
                    except:
                        lid = ''        
                    if (lid_cell.color != (174, 170, 170)) & (lid_cell.value != lid):
                        lid_cell.value = lid
                        # in weiße Farbe ändern
                        if lid_cell.color != (255, 255, 255):
                            lid_cell.color = (255, 255, 255)
                        if lid == '': # gelbes Feld
                            lid_cell.color = (255, 255, 0)

                # Eintragen der Master ID
                # Immer wieder neues Ablesen der Start- und End-ID
                global start_id, end_id
                # start_id ist ID beim Einlesen
                # start_id_new ist ID beim Klick auf Go
                # aktuelle ID ablesen und in entsprechende Spalte eintragen
                if start_row == 3:
                    akt_id = start_id_new
                else:
                    if start_id_new == start_id:            
                        if start_id in master_id_given:
                            akt_id = int(sht1.cells(start_row-1, 2).value) + 1
                        else:
                            akt_id = start_id
                    else:
                        if start_id is not None:
                            akt_id = start_id_new
                        else:
                            if start_id_new in master_id_given:
                                akt_id = int(sht1.cells(start_row-1, 2).value) + 1
                            else:
                                akt_id = start_id_new
                        start_id = start_id_new
                        end_id = end_id_new
                        
                red_flag = False
                flag_id = False
                flag_master_error = False
                for i in range(data_index_end-data_index+1):
                    # Master ID weiter hochzählen
                    cell_lid = sht1.range('D'+ str(start_row+i))
                    if cell_lid.color != (174, 170, 170):
                        if i != 0:
                            akt_id += 1
                            
                    while akt_id in master_id_given:
                        akt_id += 1
                            
                    cell = sht1.cells(start_row+i, 2)
                    cell.value = akt_id
                    cell.api.Font.Color = xw.utils.rgb_to_int((0, 0, 0))
                    # Check, ob tagesaktuelle ID überschritten wird
                    if akt_id > end_id_new:
                        cell.api.Font.Color = xw.utils.rgb_to_int((255, 0, 0))
                        red_flag = True
                        flag_master_error = True
                        last_row = start_row
                        
                    # Check, ob Master ID bereits vergeben wurde
                    if akt_id in master_id_given:
                        flag_id = True
                        cell.api.Font.Color = xw.utils.rgb_to_int((255,165,0))
                        
                if flag_master_error:
                    # Meldung, dass Master ID überschritten wurde
                    master_id_error = Label(root, text = "\nMaster ID wurde überschritten.",
                                            fg='#CD5C5C', bg = '#eeeee4',
                                            font=('Ink free',12, 'bold'))
                    master_id_error.grid(row=9, column=1, columnspan=10)                    
                    
                if flag_id:
                    id_check_error = Label(root, text = "\nMaster ID wurde schon vergeben.",
                                            fg='#CD5C5C', bg = '#eeeee4',
                                            font=('Ink free',12, 'bold'))
                    id_check_error.grid(row=9, column=1, columnspan=10)
                        
                            
                # Spalte 'Bemerkungen zum Lager' ausfüllen
                kw = heute.strftime("%U")    
                update_kw = []
                for i in range(data_index_end-data_index+1):
                    cell = sht1.range('AF' + str(start_row+i))
                    if cell.value == '/':
                        update_kw.append(cell)
                for cell in update_kw:
                    cell.value = 'KW'+kw
                    cell.color = (255, 255, 0)
                    
                
                # Dropdowns einfügen für die Spalten F, G, H, T, U, AA, AB und AD
                def return_dropdown_list(spalte):
                    dropdown_range = sht_drop.range('{}2'.format(spalte)).expand('down')
                    dropdown_values = list(filter(lambda x: pd.notna(x), dropdown_range.value))
                    return dropdown_values
                
                dropdown_g = return_dropdown_list('A')
                dropdown_h = return_dropdown_list('B')
                dropdown_i = return_dropdown_list('C')
                dropdown_u = return_dropdown_list('C')
                dropdown_m = return_dropdown_list('D')
                dropdown_v = return_dropdown_list('E')
                dropdown_ab = return_dropdown_list('B')
                dropdown_ac = return_dropdown_list('F')
                dropdown_ae = return_dropdown_list('G')
                
                # Zustand nach Prozessierung anpassen
                matrix = sht1.range('I' + str(start_row)).value
                if 'faeces' in matrix:
                    stuhl_index = dropdown_v.index('Stuhl')+1
                    dropdown_v = dropdown_v[stuhl_index:]
                elif 'urin' in matrix:
                    stuhl_index = dropdown_v.index('Stuhl')
                    urin_index = dropdown_v.index('Urin')+1
                    dropdown_v = dropdown_v[urin_index:stuhl_index]
                elif ('swab' in matrix) | ('saliv' in matrix):
                    urin_index = dropdown_v.index('Urin')
                    speichel_index = dropdown_v.index('Speichel/Swaps')+1
                    dropdown_v = dropdown_v[speichel_index:urin_index]
                else:
                    speichel_index = dropdown_v.index('Speichel/Swaps')
                    dropdown_v = dropdown_v[1:speichel_index]
                
                # Zeilen, wo Dropdowns erstellt werden sollen aus datum_index ablesen
                for index_drop in datum_index:
                    index_drop = index_drop + start_row
                    create_dropdown(spendenankunft, dropdown_g, 'G{}'.format(index_drop))
                    create_dropdown(spendenankunft, dropdown_h, 'H{}'.format(index_drop))
                    create_dropdown(spendenankunft, dropdown_i, 'I{}'.format(index_drop))
                    create_dropdown(spendenankunft, dropdown_m, 'M{}'.format(index_drop))
                    create_dropdown(spendenankunft, dropdown_u, 'U{}'.format(index_drop))
                    create_dropdown(spendenankunft, dropdown_v, 'V{}'.format(index_drop))
                    create_dropdown(spendenankunft, dropdown_ab, 'AB{}:AB{}'.format(start_row, data_index_end-data_index+start_row))
                    create_dropdown(spendenankunft, dropdown_ac, 'AC{}:AC{}'.format(start_row, data_index_end-data_index+start_row))
                    create_dropdown(spendenankunft, dropdown_ae, 'AE{}:AE{}'.format(start_row, data_index_end-data_index+start_row))

                
                # Formatierung: Umrandung
                range_line = sht1.range('B{}:AF{}'.format(start_row, data_index_end-data_index+start_row))
                range_line.api.Borders.LineStyle = LineStyle.xlContinuous
                range_line.api.Borders.Weight = 2
                # Apply thick lines as an outer border around the table
                bottom_border = range_line.api.Borders(9)  # 9 represents the bottom border
                bottom_border.LineStyle = LineStyle.xlContinuous
                bottom_border.Weight = 3
                left_border = range_line.api.Borders(7)  # 7 represents the left border
                left_border.LineStyle = LineStyle.xlContinuous
                left_border.Weight = 3
                right_border = range_line.api.Borders(10)  # 10 represents the right border
                right_border.LineStyle = LineStyle.xlContinuous
                right_border.Weight = 3

                init_id_aliquot()
                
            # Datenbank schließen
            datenbank.close()
            app.quit() 


    # Primär ID ablesen (befindet sich im Scannfeld)
    sht1 = spendenankunft.sheets['Spendenankunft']
    global prim_id_total
    prim_id_total = sht1.range('B1').value
    
    if prim_id_total is None:
        id_error = Label(root, text = "\nBitte Barcode scannen", fg='#CD5C5C',
                         bg = '#eeeee4', font=('Ink free',12, 'bold'))
        id_error.grid(row=9, column=1, columnspan=10)            
    else:    
        # Check, ob Barcode-Nr. bereits gescannt wurde
        last_row_excel = sht1.range('B3').end('down').row
        scanned_barcodes = list(sht1.range('C3:C{}'.format(last_row_excel)).value)
        scanned_barcodes = [barcode for barcode in scanned_barcodes if barcode is not None]

        if (prim_id_total in scanned_barcodes) & (red_flag_middle == False) & (red_flag == False):
            scan_text = Label(root, text = '\nDieser Barcode wurde bereits gescannt.', fg='#CD5C5C',
                                     bg = '#eeeee4', font=('Ink free',12, 'bold'))
            scan_text.grid(row=9, column=1, columnspan=10)
        else:
            # Check, ob zweite Master ID größer ist als die erste
            start_id_new = sht1.range('C1').value
            end_id_new = sht1.range('E1').value
            try:
                if start_id_new >= end_id_new:
                    end_id_error = Label(root, text = "\nBitte Master ID überprüfen", fg='#CD5C5C',
                                     bg = '#eeeee4', font=('Ink free',12, 'bold'))
                    end_id_error.grid(row=9, column=1, columnspan=10)                
                else:
                    fill_excel()
            except:
                # Meldung, wenn keine Start und End Master IDs eingegeben wurden
                if (start_id_new is None) | (end_id_new is None):
                    type_error_comp = Label(root, text = "\nBitte zu vergebene Master ID eintragen", fg='#CD5C5C',
                                         bg = '#eeeee4', font=('Ink free',12, 'bold'))
                    type_error_comp.grid(row=9, column=1, columnspan=10)
                # Meldung, wenn Barcodenr. nicht gefunden wurde
                else:
                    barcode_error = Label(root, text = "\nBarcodenr. nicht in Datenbank gefunden", fg='#CD5C5C',
                                         bg = '#eeeee4', font=('Ink free',12, 'bold'))
                    barcode_error.grid(row=9, column=1, columnspan=10)                    


        # Scannfeld löschen
        scanfield = sht1.range('B1')
        scanfield.value = ''
        # Scannfeld auswählen (über Makro)
        spendenankunft.macro("SelectCellB1")()

# Funktion zum Eintragen der Sample ID nach nachträglichem Eintragen der Aliquot-Anzahl
def enter_aliquot():
    global aliquot_error
    
    try:
        aliquot_error.grid_forget()
    except:
        pass
    
    try:
        opened_file.grid_forget()
    except:
        pass
    
    try:
        data_error.grid_forget()
    except:
        pass
    
    try:
        datenbank_error.grid_forget()
    except:
        pass
    
    try:
        id_error.grid_forget()
    except:
        pass
    
    try:
        id_check_error.grid_forget()
    except:
        pass
    
    try:
        end_id_error.grid_forget()
    except:
        pass
    
    try:
        text_error.grid_forget()
    except:
        pass
    
    try:
        type_error_comp.grid_forget()
    except:
        pass
    
    try:
        barcode_error.grid_forget()
    except:
        pass
    
    try:
        scan_text.grid_forget()
    except:
        pass
    
    def format_sample_id(sample_id):
        """ Formattiert sample ID
        """
        if len(str(sample_id)) == 1:
            sample_id = '00' + str(sample_id)
        elif len(str(sample_id)) == 2:
            sample_id = '0' + str(sample_id)
        else:
            sample_id = str(sample_id)
        return(sample_id)
    
    # Tabelle ablesen
    sht1 = spendenankunft.sheets['Spendenankunft']
    init_id_aliquot()
        
    # Zeilen ablesen, wo neue Master ID startet, da diese in der Spalte Sample ID immer mit 001 anfangen
    id_vergabe_new = [master_id for master_id in id_vergabe if type(master_id) != str]
    new_row = [0]
    for i in range(len(id_vergabe_new)-1):
        if id_vergabe_new[i] != id_vergabe_new[i+1]:
            new_row.append(i+1)
    # Liste mit letzter Zeile eines Master ID-Blocks erstellen
    last_row_id = [x-1 for x in new_row if x != 0]
    # wenn Spendenankunft nur einen Datensatz enthält, muss last_row_id ergänzt werden
    if last_row_id == []:
        last_row_id.append(len(id_vergabe_new))
    column_range = sht1.range('X{}:X{}'.format(3, 3+len(id_vergabe)))
    
    break_loop = False
    for i in range(len(aliquot_column)):
        if break_loop:
            break
        cell = sht1.cells(3+i,24)
        cell_value = cell.value
        # Meldung, wenn Aliquotzahl nicht vollständig eingetragen wurde
        if cell_value is None:
            aliquot_error = Label(root, text = "\nAliquotanzahl nicht vollständig", fg='#CD5C5C',
                                  bg = '#eeeee4', font=('Ink free',12, 'bold'))
            aliquot_error.grid(row=9, column=1, columnspan=10)            
            break
        
        cell_value = int(cell_value)
        cell_sample = sht1.cells(3+i,25)                            
        if i not in new_row:
            if cell_value == 1:
                try:
                    if len(sht1.cells(3+i-1,25).value.split('-')) == 3:
                        sample_id = int(sht1.cells(3+i-1,25).value.split('-')[-1])+1
                    else:
                        sample_id = int(sht1.cells(3+i-1,25).value.split('-')[0]) + int(sht1.cells(3+i-1,24).value)
                except: # wenn zuvor Anzahl der Aliquotzahl 0 ist
                    if i-1 in new_row:
                        sample_id = 1
                    else:
                        j = i-2
                        while True:                            
                            sample_id = sht1.cells(3+j,25).value
                            if sample_id == '':
                                j -= 1
                            else:
                                if j in new_row:
                                    sample_id = 1
                                    break
                                else:
                                    if len(sht1.cells(3+i,25).value.split('-')) == 3:
                                        sample_id = int(sht1.cells(3+j,25).value.split('-')[-1])+1
                                    else:
                                        sample_id = int(sht1.cells(3+j,25).value.split('-')[0])+1
                                    break                                
                # in Excel eintragen                    
                sample_id = format_sample_id(sample_id)
                cell_sample.value = sample_id
            elif cell_value == 0:
                sample_id = ''
                cell_sample.value = sample_id
            else:
                try:
                    if len(sht1.cells(3+i-1,25).value.split('-')) == 3:
                        sample_id = int(sht1.cells(3+i-1,25).value.split('-')[-1])+1
                    else:
                        sample_id = int(sht1.cells(3+i-1,25).value.split('-')[0])+1
                except:
                    if i-1 in new_row:
                        sample_id = 1
                    else:
                        j = i-2
                        while True:                            
                            sample_id = sht1.cells(3+j,25).value
                            if sample_id == '':
                                j -= 1
                            else:
                                if j in new_row:
                                    sample_id = 1
                                    break
                                else:
                                    if len(sht1.cells(3+i,25).value.split('-')) == 3:
                                        sample_id = int(sht1.cells(3+j,25).value.split('-')[-1])+1
                                    else:
                                        sample_id = int(sht1.cells(3+j,25).value.split('-')[0])+1
                                    break                    
                last_id = sample_id + cell_value -1                    
                sample_id = format_sample_id(sample_id)
                last_id = format_sample_id(last_id)
                # in Excel eintragen
                cell_sample.value = sample_id + '-' + last_id
        else:
            if cell_value == 1:
                sample_id = '001'
                cell_sample.value = sample_id
            elif cell_value == 0:
                sample_id = ''
                cell_sample.value = sample_id
            else:
                sample_id = 1
                last_id = sample_id + cell_value -1
                sample_id = format_sample_id(sample_id)
                last_id = format_sample_id(last_id)                    
                cell_sample.value = sample_id + '-' + last_id
        # Zelle weiß färben
        cell_sample.color = (255, 255, 255)
        cell.color = (255, 255, 255)

        # Aliquotzahl anpassen
        if i not in last_row_id:
            last_row = last_row_id[-1]
            for num in new_row:
                if num > i and num < last_row:
                    last_row = num
            for j in range(i+1,last_row): # diese Zeilen müssen auf die obere Zeile angepasst werden
                # Differenz zwischen Anzahl der alten und neuen Aliquots berechnen
                try:
                    if (sht1.cells(3+j,25).value == '') | (sht1.cells(3+j,25).value is None):
                        continue
                    else:
                        if len(sht1.cells(3+i,25).value.split('-')) == 3:
                            diff = int(sht1.cells(3+j,24).value) - int(sht1.cells(3+j,25).value.split('-')[-1])
                        else:
                            diff = int(sht1.cells(3+j,24).value)
                        # Differenz auf sample ID addieren
                        cell_sample = sht1.cells(3+j,25).value
                        if '-' not in cell_sample:
                            cell_sample = int(cell_sample) + diff
                            cell_sample = format_sample_id(cell_sample)
                            sht1.cells(3+j,25).value = cell_sample
                        else:
                            first_id = cell_sample.split('-')[0]
                            first_id = int(first_id) + diff
                            first_id = format_sample_id(first_id)
                            if len(cell_sample.split('-')) == 3:
                                last_id = cell_sample.split('-')[1]
                                last_id = int(last_id) + diff
                                last_id = format_sample_id(last_id)
                                cell_sample = first_id+'-'+last_id
                                sht1.cells(3+j,25).value = cell_sample
                            else:
                                cell_sample = first_id+'-'+str(int(first_id)+diff)
                                sht1.cells(3+j,25).value = cell_sample                                
                                
                except (ValueError, TypeError): # weil NoneType vorher nicht gecatcht wurde
                    aliquot_error = Label(root, text = "\nAliquotanzahl nicht vollständig", fg='#CD5C5C',
                                          bg = '#eeeee4', font=('Ink free',12, 'bold'))
                    aliquot_error.grid(row=9, column=1, columnspan=10)
                    break_loop = True
                    
                    

# Etiketten-Codes ausgeben
def etiketten():
    global text_error
    try:
        text_error.grid_forget()
    except:
        pass
    
    try:
        # Prüfnummer aus Master ID ablesen
        start_zeile = int(von_entry.get("1.0", "end"))
        end_zeile = int(bis_entry.get("1.0", "end"))
        # Spendenankunfttabelle von start_zeile bis end_zeile ablesen
        spendenankunft = xw.Book(path)
        sht1 = spendenankunft.sheets['Spendenankunft']
        excel_spende = sht1.used_range.options(pd.DataFrame, index=False).value
        excel_spende.columns = excel_spende.iloc[0]
        excel_spende = excel_spende[1:]
        excel_spende = excel_spende.iloc[start_zeile-3:end_zeile-2, :]
        # alle Master IDs in eine Liste wiedergeben
        master_id = list(excel_spende.iloc[:, 1])
        # Master IDs bearbeiten
        for i in range(len(master_id)):
            if str(master_id[i])[:3] != '100':
                master_id[i] = float('100' + str(master_id[i]))
        df_spende = excel_spende.copy()
        df_spende['in.vent Master ID'] = master_id
        # Aliquotnr aus Sample ID + Volumen des Gebindes ablesen
        aliquotnr = []
        gebinde = []
        for i in range(len(master_id)):
            for j in range(len(df_spende)):
                if (master_id[i] == df_spende['in.vent Master ID'].iloc[j]) | (str(master_id[i])[3:] == str(df_spende['in.vent Master ID'].iloc[j])):
                    aliquotnr.append(df_spende['Sample ID'].iloc[j].split('-')[0])
                    gebinde.append(df_spende['Gebinde'].iloc[j])
                    break
        # Add aliquotnr and verpackung as new columns
        aliquot_col = [None]*len(df_spende)
        gebinde_col = [None]*len(df_spende)
        for i in range(len(master_id)):
            for j in range(len(df_spende)):
                if (master_id[i] == df_spende['in.vent Master ID'].iloc[j]) | (str(master_id[i])[3:] == str(df_spende['in.vent Master ID'].iloc[j])):
                    aliquot_col[j] = aliquotnr[i]
                    gebinde_col[j] = gebinde[i]
        df_spende['Aliquotnr'] = aliquot_col
        df_spende['Gebinde'] = gebinde_col
        # Zeilen markieren, wo Volumen >= 250 ml ist
        for i in range(len(df_spende)):
            vol = float(df_spende['Gebinde'].iloc[i].split('(')[-1].split(' ')[0])
            if vol >= 250:
                df_spende['Lagertemperatur'].iloc[i] = str(df_spende['Lagertemperatur'].iloc[i])+'_b'

        # groupby nach Master ID, Aliquotnr und Temperatur und summieren von Anzahl der Aliquots
        df_spende['Anzahl der Aliquots'] = df_spende['Anzahl der Aliquots'].map(lambda x: int(x))
        df_spende = df_spende.groupby(['in.vent Master ID', 'Aliquotnr', 'Lagertemperatur', 'Gebinde']).sum()['Anzahl der Aliquots']
        df_spende = df_spende.reset_index()
        # swap columns
        df_spende['Temperatur'] = df_spende['Lagertemperatur']
        df_spende = df_spende.drop('Lagertemperatur', axis=1)
        # Temperatur-Spalte überarbeiten
        for i in range(len(df_spende)):
            if 'degrees Celsius' in df_spende['Temperatur'].iloc[i]:
                temp = df_spende['Temperatur'].iloc[i].replace('degrees Celsius', '°C')
                temp = ''.join(temp.strip(' '))
                df_spende.loc[i, 'Temperatur'] = temp

        # 001-Etiketten anhand Volumen des Gebindes trennen
        for i in range(len(df_spende)):
            vol = float(df_spende['Gebinde'].iloc[i].split('(')[-1].split(' ')[0])
            if vol >= 250:
                df_spende.loc[i, 'Gebinde'] = 'gross'

            else:
                df_spende.loc[i, 'Gebinde'] = 'klein'
        df_001 = df_spende[df_spende['Aliquotnr']=='001']
        df_001 = df_001[df_001['Gebinde']=='gross']
        drop_rows = []
        for i in range(len(df_001)):
            if 'b' not in df_001['Temperatur'].iloc[i]:
                drop_rows.append(i)
            if 'b' in df_001['Temperatur'].iloc[i]:
                df_001['Temperatur'].iloc[i] = df_001['Temperatur'].iloc[i].replace('_b', '')
        try:
            df_001 = df_001.drop(drop_rows)
            df_001 = df_001.reset_index().drop('index', axis=1)
        except:
            pass
        
        df_001 = df_001.groupby(['in.vent Master ID', 'Aliquotnr', 'Gebinde','Temperatur']).sum()['Anzahl der Aliquots']
        df_001 = df_001.reset_index()
        df_rest_gr = df_001.copy()
        df_rest_gr['Anzahl der Aliquots'] = df_001['Anzahl der Aliquots']-1
        df_rest_gr['Aliquotnr'] = '002'
        df_001['Anzahl der Aliquots'] = 1

        # restl. Etiketten
        # zuerst Marker 'b' entfernen
        for i in range(len(df_spende)):
            if 'b' in df_spende['Temperatur'].iloc[i]:
                df_spende['Temperatur'].iloc[i] = df_spende['Temperatur'].iloc[i].replace('_b', '')
        df_rest = df_spende.merge(df_001, how='outer', indicator=True).loc[lambda x: x['_merge'] == 'left_only']
        df_rest.drop('_merge', axis=1, inplace=True)
        # df_rest weiter ergänzen
        # wenn Master ID in df_rest und df_001 ist, dann ist Aliquotnr in df_rest = 2
        masterid_001 = list(df_001['in.vent Master ID'])
        for i in range(len(df_rest)):
            if df_rest['in.vent Master ID'].iloc[i] in masterid_001:
                df_rest['Aliquotnr'].iloc[i] = '002'

        # dfs zusammenfügen
        # aus df_rest große Zeilen entfernen, wo Gebinde groß ist, da diese in df_rest_gr sind
        df_rest = df_rest[df_rest['Gebinde']=='klein']
        df_rest = pd.concat([df_rest, df_rest_gr]).reset_index().drop('index', axis=1)
        # sort df_rest
        df_rest = df_rest.sort_values(by=df_rest.columns[0], axis=0)
        # Gebinde-Spalte entfernen
        df_001 = df_001.drop('Gebinde', axis = 1)
        df_rest = df_rest.drop('Gebinde', axis = 1)
        # reorder columns of df_001
        df_001 = df_001.loc[:,['in.vent Master ID', 'Aliquotnr', 'Anzahl der Aliquots', 'Temperatur']]
        # Nuller-Etiketten erstellen
        # Temperatur für Proben in df_001 übernehmen
        df_000 = pd.DataFrame({
            'in.vent Master ID':[],
            'Aliquotnr':[],
            'Anzahl der Aliquots':[],
            'Temperatur':[]
        })
        id_001 = []
        if len(df_001) != 0:
            df_000 = df_001.copy()
            df_000['Aliquotnr'] = '000'
            id_001 = list(df_001['in.vent Master ID'])
        # bei restl. Proben: T nach Prio RT > -70 > 2-8 > -18
        t_prio = ['room temperature', '-70', '2-8', '-18']
        df_000_rest = df_rest.copy()
        # reset index
        df_000_rest = df_000_rest.reset_index()
        drop_index = []
        for i in range(len(df_rest)):
            if df_rest['in.vent Master ID'].iloc[i] in id_001:
                drop_index.append(i)

        if drop_index != []:
            df_000_rest = df_000_rest.drop(drop_index, axis=0)
        df_000_rest['Aliquotnr'] = '000'
        df_000_rest['Prio'] = df_000_rest['Temperatur']
        
        for i in range(len(df_000_rest)):
            for j in range(len(t_prio)):
                if t_prio[j] in df_000_rest['Prio'].iloc[i].lower():
                    df_000_rest.loc[i, 'Prio'] = str(j)
        
        df_000_rest['Prio'] = df_000_rest['Prio'].map(int)
        df_000_rest = df_000_rest.reset_index().drop('index', axis=1)
        # keep row with the highest prio
        index_same_id = []
        df_000_rest = df_000_rest.sort_values('Prio')
        df_000_rest = df_000_rest.drop_duplicates('in.vent Master ID', keep='first')
        df_000_rest = df_000_rest.drop('Prio', axis=1)
        
        df_000 = pd.concat([df_000, df_000_rest]).sort_values('in.vent Master ID')
        df_000['Anzahl der Aliquots'] = 1
        
        # insert all the tables df_000, df_001 and df_rest into the excel sheet
        # open 'Etiketten-code' file
        file_path = r'O:\Etiketten\Etiketten-code.xls'

        check = pd.read_excel(file_path, sheet_name = 'SS Aliquot ID')
        check_ma = pd.read_excel(file_path, sheet_name = 'Master Aliquot ID ')
        wb = xw.Book(file_path)
        ws = wb.sheets['SS Aliquot ID']
        start_row = len(check) +3
        ws.range(f'A{start_row}').options(index=False, header=False).value = df_000
        start_row += len(df_000) +1
        ws.range(f'A{start_row}').options(index=False, header=False).value = df_rest

        check_groß = pd.read_excel(file_path, sheet_name = 'Master Aliquot ID ')
        ws_groß = wb.sheets['Master Aliquot ID ']
        start_row = len(check_ma) +3
        ws_groß.range(f'A{start_row}').options(index=False, header=False).value = df_001        
        
    # falls Aliquotanzahl nicht ausgefüllt wurde
    except:
        import traceback
        traceback.print_exc()
        text_error = Label(root, text = "\nAnzahl der Aliquots noch nicht ergänzt", fg='#CD5C5C',
                     bg = '#eeeee4', font=('Ink free',12, 'bold'))
        text_error.grid(row=9, column=1, columnspan=10)



In [5]:
# User Interface:

# Basis
root = Tk()
root.title('AutoSpendenankunft')
root.geometry('500x350')
root.iconbitmap('O:\Forschung & Entwicklung\Allgemein\Vorlagen\Abbildungen\Dodo\dodo_icon.ico')
root.config(bg='#eeeee4')

# alle Buttons auf Startseite
hi = Label(root, text = '\nAutoSpendenankunft\n', bg = '#eeeee4', font=('Ink free',15,'bold'))
hi.grid(row=1, column=1, columnspan=5)

browse_text = Label(root, text = 'Bitte wähle die Tabelle aus.', bg = '#eeeee4', font=('Ink free',12))
browse_text.grid(row=2, column=1, columnspan=5)
browse_button = Button(root, text='Browse', command=browse, font=('Ink free',10,'bold'), bg='#869287')
browse_button.grid(row=2, column=6)
space = Label(root, text = '', bg = '#eeeee4', font=('Ink free',12))
space.grid(row=3, column=1)

barcode_text = Label(root, text = 'Barcode gescannt?', bg = '#eeeee4', font=('Ink free',12))
barcode_text.grid(row=4, column=1, columnspan=5)
barcode_button = Button(root, text='   Go   ', font=('Ink free',10,'bold'), command=go_dodo, bg='#869287')
barcode_button.grid(row=4, column=6)

aliquot_text = Label(root, text = 'Sample ID vergeben', bg = '#eeeee4', font=('Ink free',12))
aliquot_text.grid(row=5, column=1, columnspan=5)
aliquot_button = Button(root, text='   Go   ', font=('Ink free',10,'bold'), command=enter_aliquot, bg='#869287')
aliquot_button.grid(row=5, column=6)

space2 = Label(root, text = '', bg = '#eeeee4', font=('Ink free',12))
space2.grid(row=6, column=1)

codes_text = Label(root, text = 'Etikettencode generieren        ',
                   bg = '#eeeee4', font=('Ink free',12))
codes_text.grid(row=7, column=1, columnspan=20)
von_text = Label(root, text = 'von', bg = '#eeeee4', font=('Ink free',12))
von_text.grid(row=8, column=1)
von_entry = Text(root, width=15, height=1)
von_entry.config(font=('Ink free',10))
von_entry.grid(row=8, column=2)
bis_text = Label(root, text = 'bis', bg = '#eeeee4', font=('Ink free',12))
bis_text.grid(row=8, column=3)
bis_entry = Text(root, width=15, height=1)
bis_entry.config(font=('Ink free',10))
bis_entry.grid(row=8, column=4)
codes_button = Button(root, text='   Go   ', font=('Ink free',10,'bold'), command=etiketten, bg='#869287')
codes_button.grid(row=8, column=6)


# Logo
from PIL import ImageTk, Image
frame = Frame(root, width=1, height=1)
frame.grid(row=1, column=0)

img = ImageTk.PhotoImage(Image.open("O:\Forschung & Entwicklung\Allgemein\Vorlagen\Abbildungen\Dodo\dodo-dancing_ohne Hintergrund_ohne Schatten.png").resize((70,70)), master = root)
label = Label(frame, image = img, bg = '#eeeee4')
label.pack()

# kopierte Datenbank wird beim Schließen des GUIs gelöscht
def delete_copied_file():
    try:
        os.remove('Datenbank_Copy.xlsx')
    except:
        pass
    root.destroy()
    
root.protocol("WM_DELETE_WINDOW", delete_copied_file)


root.mainloop()


In [6]:
#1004-231356N-001

# Sub CreateDropdown(dropdownArray As Variant, targetCell As String)
#     With ThisWorkbook.Sheets("Spendenankunft").Range(targetCell).Validation
#         .Delete
#         .Add Type:=xlValidateList, AlertStyle:=xlValidAlertStop, Operator:= _
#             xlBetween, Formula1:=Join(dropdownArray, ",")
#     End With
# End Sub